# Concatenate low mass range and high mass range datasets

In [ ]:
from pathlib import Path
from concatenate_imzml_files import concat_imzml_files
from definitions import ROOT_DIR
import numpy as np

Provide inputs
- Folder where imzmls for all matrices are located

In [ ]:
p = Path(ROOT_DIR) / "1_stitch_and_upload_datasets"

- Iterate over files to find pairs that will be stitched

In [ ]:
all_args = []

for mpath in p.iterdir():
    if mpath.is_dir():
        fname_pos = mpath / f"{mpath.name}_pos_mz70-1510.imzML"
        lo_pos = list(mpath.rglob('*pos*mz70*.imzML'))[0]
        hi_pos = list(mpath.rglob('*pos*mz300*.imzML'))[0]
        
        fname_neg = mpath / f"{mpath.name}_neg_mz70-1510.imzML" 
        lo_neg = list(mpath.rglob('*neg*mz70*.imzML'))[0]
        hi_neg = list(mpath.rglob('*neg*mz300*.imzML'))[0]
        
        all_args.append([[lo_pos, hi_pos], fname_pos])
        all_args.append([[lo_neg, hi_neg], fname_neg])
        
np.save(p / 'stitching_inputs.npy', list(all_args))

- Concatenate each pair of files

In [ ]:
for input_paths, output_path in all_args:
    offsets = [(0,0), (0,0)] # (x,y) tuple for each imzML file if you want to offset either file's coordinates
    mz_ranges = [('auto','auto'), ('auto','auto')] # (min_mz, max_mz) tuples. Replace 'auto' with a non-string number if you want to explicitly specify the m/z range for each file
    concat_imzml_files(input_paths, offsets, mz_ranges, output_path)

# Upload stitched datasets to METASPACE

In [1]:
from pathlib import Path
import json, getpass
from metaspace import SMInstance
import pandas as pd

- To avoid processing with HMDB, upload on staging

In [2]:
sm = SMInstance(host='https://staging.metaspace2020.eu')

if not sm.logged_in():
    # Using getpass here prevents the API key from being accidentally saved with this notebook.
    api_key = getpass.getpass(prompt='API key: ', stream=None)
    sm.login(api_key=api_key)

API key: ········


Provide inputs, for each pair:

matrix full name, additional neutral gain just for this matrix (matrix molecule), solvent

In [3]:
matrix_fullname = dict({
    'DHB':['2,5-dihydroxybenzoic acid', ['+C7H6O4'], 'ACN (70% v/v, aq.)'],
    'DAN':['1,5-diaminonaphthalene', ['+C10H10N2', '+C10H8N2'], 'ACN (70% v/v, aq.)'],
    'norharmane':['norharmane', ['+C11H8N2'], 'CHCl3:MeOH (1:1 v/v)'],
    '9AA':['9-aminoacridine', ['+C13H10N2'], 'MeOH (70% v/v, aq.)'],
    'CHCA':['alpha-cyano-4-hydroxycinnamic acid', ['+C10H7NO3'], 'ACN (50% v/v, aq.)'],
    'ClCCA':['4-chloro-alpha-cyanocinnamic acid', ['+C10H6ClNO2'], 'ACN (50% v/v, aq.)'],
    'NEDC':['N-(1-naphthyl)ethylenediamine dihydrochloride', ['+C12H14N2', '+HCl'], 'ACN (70% v/v, aq.)'],
    'PNDIT2':['PNDI-T2', [], 'Toluene'],
    'MAPS':['Maleic anhydride proton sponge', ['+C18H18N2O3'], 'Toluene'],
    'DHAP': ['2,5-dihydroxyacetophenone', ['+C8H8O3'], 'ACN (70% v/v, aq.)']
})

- Define neutral losses in the function below

Might be of interest for interpreting neutral losses http://www.colby.edu/chemistry/PChem/StableLoss.html

In [4]:
def submit_dataset(dataset_name, matrix, solvent, polarity, adducts, extra_neutral_losses = [], databases = [('Spotting_project_compounds-v9', 'feb2021')], is_public = False):

    metadata = {
        'Data_Type': 'Imaging MS',  # shouldn't be changed
        'Sample_Information': {
            'Organism': 'None',
            'Organism_Part': 'None',
            'Condition': 'None',
            'Sample_Growth_Conditions': 'None'  # this is an extra field
        },
        'Sample_Preparation': {
            'Sample_Stabilisation': 'None',
            'Tissue_Modification': 'None',
            'MALDI_Matrix': matrix,
            'MALDI_Matrix_Application': 'TM-Sprayer',
            'Solvent': solvent,
        },
        'MS_Analysis': {
            'Polarity': polarity,
            'Ionisation_Source': 'AP-SMALDI5',
            'Analyzer': 'Orbitrap',
            'Detector_Resolving_Power': {
                'mz': 200,
                'Resolving_Power': 140000
            },
            'Pixel_Size': {
                'Xaxis': 150,
                'Yaxis': 150
            }
        }
    }
    
    ds_id  = sm.submit_dataset(
    imzml_fn, ibd_fn, dataset_name,
    json.dumps(metadata), is_public, databases,
    project_ids=['60b51b5a-af5a-11eb-b5f0-fb6f98d16f5b'], #staging
    adducts=adducts,
    neutral_losses = ['-H2O', '-H2', '+H2', #redox
                      '-CO2', '-CH2O3', '-CH2O2', # CO2+H2O, formic acid?
                      '-HPO3', '-H3PO4', # phosphate
                      '-NH3', '-C2H5NO2',  # glycine
                     ] + extra_neutral_losses
        
    )
    
    return ds_id

- Upload datasets

In [5]:
p = Path(r'D:\saharuka\spotting\20_matrices')

matrices = []
modes = []
ds_names = []
ids = []
losses = []

for mpath in p.iterdir():
    if mpath.is_dir():
        for pol in ['pos', 'neg']:
            imzml_fn = list(mpath.rglob(f"*{pol}*mz70-1510*.imzML"))[0]
            ibd_fn = list(mpath.rglob(f"*{pol}*mz70-1510*.ibd"))[0]
            dataset_name = imzml_fn.name
            matrix = matrix_fullname[mpath.name][0]
            extra_neutral_losses = matrix_fullname[mpath.name][1]
            solvent = matrix_fullname[mpath.name][2]
            
            if pol == 'pos':
                polarity = 'Positive'
                adducts = ['[M]+', '+H', '+Na', '+K']
            else: 
                polarity = 'Negative'
                adducts = ['[M]-', '-H', '+Cl']
            
            ds_id = submit_dataset(dataset_name, matrix, solvent, polarity, adducts, extra_neutral_losses)

            matrices.append(matrix)
            modes.append(polarity)
            ds_names.append(dataset_name)
            ids.append(ds_id)
            losses.append(extra_neutral_losses)
            
df = pd.DataFrame({
    'matrix':matrices,
    'polarity':modes,
    'ds_name':ds_names,
    'ds_id':ids,
    'extra_neutral_losses':losses
})

Uploading   1 part of 9AA_pos_mz70-1510.imzML file...
Uploading   2 part of 9AA_pos_mz70-1510.imzML file...
Uploading   3 part of 9AA_pos_mz70-1510.imzML file...
Uploading   4 part of 9AA_pos_mz70-1510.imzML file...
Uploading   5 part of 9AA_pos_mz70-1510.imzML file...
Uploading   6 part of 9AA_pos_mz70-1510.imzML file...
Uploading   7 part of 9AA_pos_mz70-1510.imzML file...
Uploading   8 part of 9AA_pos_mz70-1510.imzML file...
Uploading   9 part of 9AA_pos_mz70-1510.imzML file...
Uploading  10 part of 9AA_pos_mz70-1510.imzML file...
Uploading  11 part of 9AA_pos_mz70-1510.imzML file...
Uploading  12 part of 9AA_pos_mz70-1510.imzML file...
Uploading  13 part of 9AA_pos_mz70-1510.imzML file...
Uploading  14 part of 9AA_pos_mz70-1510.imzML file...
Uploading  15 part of 9AA_pos_mz70-1510.imzML file...
Uploading  16 part of 9AA_pos_mz70-1510.imzML file...
Uploading  17 part of 9AA_pos_mz70-1510.imzML file...
Uploading   1 part of 9AA_pos_mz70-1510.ibd file...
Uploading   2 part of 9AA_pos_

Uploading  17 part of 9AA_neg_mz70-1510.imzML file...
Uploading   1 part of 9AA_neg_mz70-1510.ibd file...
Uploading   2 part of 9AA_neg_mz70-1510.ibd file...
Uploading   3 part of 9AA_neg_mz70-1510.ibd file...
Uploading   4 part of 9AA_neg_mz70-1510.ibd file...
Uploading   5 part of 9AA_neg_mz70-1510.ibd file...
Uploading   6 part of 9AA_neg_mz70-1510.ibd file...
Uploading   7 part of 9AA_neg_mz70-1510.ibd file...
Uploading   8 part of 9AA_neg_mz70-1510.ibd file...
Uploading   9 part of 9AA_neg_mz70-1510.ibd file...
Uploading  10 part of 9AA_neg_mz70-1510.ibd file...
Uploading  11 part of 9AA_neg_mz70-1510.ibd file...
Uploading  12 part of 9AA_neg_mz70-1510.ibd file...
Uploading  13 part of 9AA_neg_mz70-1510.ibd file...
Uploading  14 part of 9AA_neg_mz70-1510.ibd file...
Uploading  15 part of 9AA_neg_mz70-1510.ibd file...
Uploading  16 part of 9AA_neg_mz70-1510.ibd file...
Uploading  17 part of 9AA_neg_mz70-1510.ibd file...
Uploading  18 part of 9AA_neg_mz70-1510.ibd file...
Uploading 

Uploading 158 part of 9AA_neg_mz70-1510.ibd file...
Uploading 159 part of 9AA_neg_mz70-1510.ibd file...
Uploading 160 part of 9AA_neg_mz70-1510.ibd file...
Uploading 161 part of 9AA_neg_mz70-1510.ibd file...
Uploading 162 part of 9AA_neg_mz70-1510.ibd file...
Uploading 163 part of 9AA_neg_mz70-1510.ibd file...
Uploading 164 part of 9AA_neg_mz70-1510.ibd file...
Uploading 165 part of 9AA_neg_mz70-1510.ibd file...
Uploading 166 part of 9AA_neg_mz70-1510.ibd file...
Uploading 167 part of 9AA_neg_mz70-1510.ibd file...
Uploading 168 part of 9AA_neg_mz70-1510.ibd file...
Uploading 169 part of 9AA_neg_mz70-1510.ibd file...
Uploading 170 part of 9AA_neg_mz70-1510.ibd file...
Uploading 171 part of 9AA_neg_mz70-1510.ibd file...
Uploading 172 part of 9AA_neg_mz70-1510.ibd file...
Uploading 173 part of 9AA_neg_mz70-1510.ibd file...
Uploading 174 part of 9AA_neg_mz70-1510.ibd file...
Uploading 175 part of 9AA_neg_mz70-1510.ibd file...
Uploading 176 part of 9AA_neg_mz70-1510.ibd file...
Uploading 17

Uploading  10 part of CHCA_pos_mz70-1510.imzML file...
Uploading  11 part of CHCA_pos_mz70-1510.imzML file...
Uploading  12 part of CHCA_pos_mz70-1510.imzML file...
Uploading  13 part of CHCA_pos_mz70-1510.imzML file...
Uploading  14 part of CHCA_pos_mz70-1510.imzML file...
Uploading  15 part of CHCA_pos_mz70-1510.imzML file...
Uploading  16 part of CHCA_pos_mz70-1510.imzML file...
Uploading  17 part of CHCA_pos_mz70-1510.imzML file...
Uploading   1 part of CHCA_pos_mz70-1510.ibd file...
Uploading   2 part of CHCA_pos_mz70-1510.ibd file...
Uploading   3 part of CHCA_pos_mz70-1510.ibd file...
Uploading   4 part of CHCA_pos_mz70-1510.ibd file...
Uploading   5 part of CHCA_pos_mz70-1510.ibd file...
Uploading   6 part of CHCA_pos_mz70-1510.ibd file...
Uploading   7 part of CHCA_pos_mz70-1510.ibd file...
Uploading   8 part of CHCA_pos_mz70-1510.ibd file...
Uploading   9 part of CHCA_pos_mz70-1510.ibd file...
Uploading  10 part of CHCA_pos_mz70-1510.ibd file...
Uploading  11 part of CHCA_pos

Uploading 148 part of CHCA_pos_mz70-1510.ibd file...
Uploading 149 part of CHCA_pos_mz70-1510.ibd file...
Uploading 150 part of CHCA_pos_mz70-1510.ibd file...
Uploading 151 part of CHCA_pos_mz70-1510.ibd file...
Uploading 152 part of CHCA_pos_mz70-1510.ibd file...
Uploading 153 part of CHCA_pos_mz70-1510.ibd file...
Uploading 154 part of CHCA_pos_mz70-1510.ibd file...
Uploading 155 part of CHCA_pos_mz70-1510.ibd file...
Uploading 156 part of CHCA_pos_mz70-1510.ibd file...
Uploading 157 part of CHCA_pos_mz70-1510.ibd file...
Uploading 158 part of CHCA_pos_mz70-1510.ibd file...
Uploading 159 part of CHCA_pos_mz70-1510.ibd file...
Uploading 160 part of CHCA_pos_mz70-1510.ibd file...
Uploading 161 part of CHCA_pos_mz70-1510.ibd file...
Uploading 162 part of CHCA_pos_mz70-1510.ibd file...
Uploading 163 part of CHCA_pos_mz70-1510.ibd file...
Uploading 164 part of CHCA_pos_mz70-1510.ibd file...
Uploading 165 part of CHCA_pos_mz70-1510.ibd file...
Uploading 166 part of CHCA_pos_mz70-1510.ibd f

...

Uploading 222 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 223 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 224 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 225 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 226 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 227 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 228 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 229 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 230 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 231 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 232 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 233 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 234 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 235 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 236 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 237 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 238 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 239 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 

Uploading 371 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 372 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 373 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 374 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 375 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 376 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 377 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 378 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 379 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 380 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 381 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 382 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 383 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 384 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 385 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 386 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 387 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 388 part of PNDIT2_pos_mz70-1510.ibd file...
Uploading 

Uploading  18 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  19 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  20 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  21 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  22 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  23 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  24 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  25 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  26 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  27 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  28 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  29 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  30 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  31 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  32 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  33 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  34 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading  35 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 

Uploading 167 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 168 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 169 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 170 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 171 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 172 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 173 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 174 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 175 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 176 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 177 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 178 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 179 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 180 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 181 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 182 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 183 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 184 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 

Uploading 316 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 317 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 318 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 319 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 320 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 321 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 322 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 323 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 324 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 325 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 326 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 327 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 328 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 329 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 330 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 331 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 332 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 333 part of PNDIT2_neg_mz70-1510.ibd file...
Uploading 

- Save information about uploaded dataset including dataset id on staging

In [6]:
df.to_csv(p / 'staging_uploaded_datasets.csv')